In [1]:
import lib.datautility as du
import numpy as np
import lib.tf_network2 as tfnet
import lib.evaluationutility as eu
from lib.tf_network2 import Network, Cost, Normalization, Optimizer
import tensorflow as tf
import session_info

/home/assistments/anaconda3/envs/reloop/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
import zipfile
with zipfile.ZipFile('resources/resources.zip','r') as zf:
    zf.extractall('resources/')

In [3]:
data, headers = du.read_csv('resources/remnant.csv')
du.print_descriptives(data, headers)

data, headers = du.read_csv('resources/experimental.csv')
du.print_descriptives(data, headers)

-- loading resources/remnant.csv...(100%)   
DESCRIPTIVES    569701 Rows, 33 Columns 
0 is_experiment (int):         0 missing (0.0%)     mean=0.21 (SD=0.41)                                                                              
1 user_id (int):               0 missing (0.0%)     mean=246611.33 (SD=114557.45)                                                                    
2 assignment_id (int):         0 missing (0.0%)     mean=989208.69 (SD=535439.25)                                                                    
3 sequence_id (int):           0 missing (0.0%)     mean=76258.05 (SD=164464.55)                                                                     
4 start_time (string):         18697 missing (0.0%) 36775 unique values                                                                              
5 end_time (string):           106477 missing (0.2%) 36202 unique values                                                                              
6 ex_sequence

19 future_ex (int):            0 missing (0.0%)     mean=2.09 (SD=1.57)                                                                              
20 is_test (int):              0 missing (0.0%)     mean=1.00 (SD=0.00)                                                                              
21 root_problem... (float):    0 missing (0.0%)     mean=2.16 (SD=0.74)                                                                              
22 root_problem... (float):    0 missing (0.0%)     mean=2.10 (SD=0.83)                                                                              
23 root_inverse... (float):    0 missing (0.0%)     mean=0.44 (SD=0.19)                                                                              
24 percent_corr... (float):    2657 missing (0.0%)  mean=0.78 (SD=0.25)                                                                              
25 root_percent... (float):    2657 missing (0.0%)  mean=0.86 (SD=0.21)                             

In [4]:
seq = dict()

key = [1]
label = [[13], [15], [12]]
cov = [7, 8, 11, 12, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
iden = [0, 2, 3, 6]
sortby = [4]

seq = tfnet.format_sequence_from_file('resources/remnant.csv',key,label,cov,iden,sortby)
print('formatting identifiers...')
seq['key'] = tfnet.fold_by_key(seq['key'], -1, 10)
print('formatting output labels...')
seq['y'] = tfnet.offset_multi_label(seq['y'],2,-1)
print('formatting feature columns...')
seq['x'] = tfnet.sequence_impute_missing(seq['x'])
print('done!')

-- loading resources/remnant.csv...(100%)   
-- preparing to format sequence from file...
-- formatting sequence...(0.75%) 

/home/assistments/Desktop/rebarLoop/Python Code/Study_1/lib/tf_network2.py:2001: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  seq['x'] = np.array(x)
/home/assistments/Desktop/rebarLoop/Python Code/Study_1/lib/tf_network2.py:2003: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  seq['iden'] = np.array(id)


-- formatting sequence...(100%)   


/home/assistments/anaconda3/envs/reloop/lib/python3.7/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


formatting identifiers...
formatting output labels...
formatting feature columns...
done!


In [5]:
seqt = dict()

key = [33, 1]
label = [[13], [15], [12]]
cov = [7, 8, 11, 12, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
iden = [38, 34, 2, 3]
sortby = [4]

tf.compat.v1.disable_eager_execution()

seqt = tfnet.format_sequence_from_file('resources/experimental.csv', key, label, cov, iden, sortby)
print('formatting output labels...')
seqt['y'] = tfnet.offset_multi_label(seqt['y'], 2, -1)
seqt['y'] = tfnet.extract_from_multi_label(seqt['y'], [0, 1])
print('formatting feature columns...')
seqt['x'] = tfnet.sequence_impute_missing(seqt['x'])
print('done!')

-- loading resources/experimental.csv...(100%)   
-- preparing to format sequence from file...
-- formatting sequence...(100%)   
formatting output labels...
formatting feature columns...
done!


In [6]:
max_epochs = 200
hidden = 50
batch = 64
keep = .5
step = 5e-4
threshold = .001
optimizer = Optimizer.ADAM

tf.compat.v1.disable_eager_execution()

n_cov = len(seq['x'][0][0])
seq['y'] = tfnet.extract_from_multi_label(seq['y'], [0, 1])
desc = tfnet.describe_multi_label(seq['y'], True)

exp = np.unique(seqt['key'].reshape((-1, len(key)))[:, 0])

======  Label Description  =============
-- Number of Samples: 130678
-- Number of Label Sets: 2
---- 1: 1 Label
---- 2: 1 Label



In [7]:
tf.compat.v1.reset_default_graph()
tf.compat.v1.set_random_seed(1)
np.random.seed(1)

# build the model
net = Network('study1_model').add_input_layer(n_cov, normalization=Normalization.Z_SCORE)
net.add_lstm_layer(hidden, peepholes=True, activation=tf.nn.leaky_relu)
net.begin_multi_output(cost_methods=[Cost.BINARY_CROSS_ENTROPY,
                                     Cost.RMSE])
net.add_dropout_layer(1, keep=keep, activation=tf.nn.sigmoid)
net.add_dropout_layer(1, keep=keep, activation=tf.identity)
net.end_multi_output()

# set defaults
net.set_default_cost_method(Cost.BINARY_CROSS_ENTROPY)
net.set_optimizer(optimizer)

# train the model and save the trained weights
net.train(x=seq['x'],
          y=seq['y'],
          step=step,
          use_validation=True,
          max_epochs=max_epochs, threshold=threshold, batch=batch)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
WARNING - Incomplete validation set information was provided. Using default 70/30% split.
            Training Network            
-3 layers: 16n -> 50n -> 1n, 1n
-200 epochs
-step size = 0.0005
-batch size = 64
Epoch        Cost    Val Cost   Delta         Time


/home/assistments/Desktop/rebarLoop/Python Code/Study_1/lib/tf_network2.py:1048: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array([a[i] for a in series_label])]).reshape((len(series_batch), n_timestep, -1)))


Epoch 1     0.9853    0.7238    0.0000      162.6s
Epoch 2     0.9092    0.7212    0.0013      166.8s
Epoch 3     0.9018    0.7166    0.0019      170.3s
Epoch 4     0.8976    0.7139    0.0016      194.1s
Epoch 5     0.8960    0.7131    0.0012      191.5s
Epoch 6     0.8942    0.7129    0.0022      204.8s
Epoch 7     0.8930    0.7118    0.0019      201.7s
Epoch 8     0.8918    0.7114    0.0010      189.3s
Epoch 9     0.8911    0.7114    0.0005      192.6s
Total Time: 1673.7s


In [8]:
net.save_model_weights()      # comment to use a previously-saved model
#
# net.build()                 # uncomment to use previously-saved model
# net.restore_model_weights() # uncomment to use previously-saved model

In [9]:
for e in range(len(exp)):
    print('formatting model predictions for experiment {}'.format(exp[e]))
    test = np.argwhere(seqt['key'].reshape((-1, len(key)))[:, 0] == exp[e]).ravel()

    pred = net.predict(x=seqt['x'][test], batch=1024)

    fold_pred = tfnet.flatten_sequence(seqt['y'][test], key=seqt['key'].reshape((-1, len(key)))[test], identifier=seqt['iden'][test])

    for p in range(len(pred)):
        tr_max = 1
        tr_min = 0
        upperbound = 1 if p == 0 else 0.333

        # and apply it to the experimental predictions
        fold_pred = np.append(fold_pred,
                              ((tfnet.flatten_sequence(pred[p]) - tr_min) / (tr_max - tr_min)) * upperbound,
                              axis=1)

    hdr = np.array(headers)[iden]
    hdr = np.append(hdr, np.array(headers)[key])

    hdr = np.append(hdr, ['complete', 'inv_mastery', 'p_complete', 'p_inv_mastery'])
    du.write_csv(fold_pred, 'model_predictions.csv', hdr if e == 0 else None, append=e > 0)
print('done!')

formatting model predictions for experiment 226210


/home/assistments/Desktop/rebarLoop/Python Code/Study_1/lib/tf_network2.py:1428: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pred.append(np.array(out_p))
/home/assistments/Desktop/rebarLoop/Python Code/Study_1/lib/tf_network2.py:1434: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pred[i] = np.append(pred[i], np.array(out_p))


formatting model predictions for experiment 237447
formatting model predictions for experiment 241501
formatting model predictions for experiment 241622
formatting model predictions for experiment 243393
formatting model predictions for experiment 246482
formatting model predictions for experiment 246627
formatting model predictions for experiment 246647
formatting model predictions for experiment 250476
formatting model predictions for experiment 255116
formatting model predictions for experiment 256017
formatting model predictions for experiment 256027
formatting model predictions for experiment 259379
formatting model predictions for experiment 263015
formatting model predictions for experiment 263052
formatting model predictions for experiment 263057
formatting model predictions for experiment 263109
formatting model predictions for experiment 263115
formatting model predictions for experiment 293151
formatting model predictions for experiment 303899
formatting model predictions fo

In [10]:
import graph_def_editor as graph_editor
import lib.datautility as du
import warnings
import time
import csv
import sys
import os
import pandas as pd

session_info.show()

In [11]:
net.session.close()